In [15]:
%pip install openai


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


1.83.0


In [24]:
from openai import OpenAI

client = OpenAI()

def router_agent(user_input: str) -> str:
    if any(word in user_input.lower() for word in ["code", "function", "bug", "implement"]):
        return "coding"
    if any(word in user_input.lower() for word in ["explain", "what is", "how does"]):
        return "research"
    return "general"


def coding_agent(task: str) -> str:
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=f"You are a coding agent. Solve this:\n{task}",
    )
    return response.output_text

def research_agent(task: str) -> str:
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=f"You are a research agent. Explain clearly:\n{task}",
    )
    return response.output_text

def swarm_run(user_input: str):
    route = router_agent(user_input)

    if route == "coding":
        return coding_agent(user_input)

    if route == "research":
        return research_agent(user_input)

    return "No suitable agent found."

print(
    swarm_run("Write a Python function to debounce an API call")
)


Certainly! Debouncing an API call means ensuring that the function is only executed once after a certain period of inactivity, even if it's invoked multiple times rapidly. This is useful to avoid excessive calls, e.g., when typing input that triggers an API call.

Here's a simple Python example using `asyncio` to debounce an async API call:

```python
import asyncio
from typing import Callable, Any, Optional

def debounce(wait: float):
    """
    Decorator that debounces an async function so that it runs only
    after `wait` seconds have passed without another call.
    """
    def decorator(func: Callable[..., asyncio.Future]):
        task: Optional[asyncio.Task] = None
        last_args = None
        last_kwargs = None

        async def debounced(*args, **kwargs):
            nonlocal task, last_args, last_kwargs
            last_args = args
            last_kwargs = kwargs
            
            # Cancel the previous delayed call if it exists
            if task and not task.